## Churn prediction

This notebook analyzes the customer data from a telecommunications company and, based on the features of the clients that unsubscribe from the previous month, it will try to predict which clients will unsubscribe this month.

It could be very useful to manage market campaigns to those clients with a high probability of unsubscribe.

### Extract, clean and transform the data

In [3]:
#Start by importing the needed packages

import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


For each month, we have four files with the following information: 
* customers_month
    1. **id**.- id of the client
    2. **age**.- age of the client
    3. **billing**.- money the client have spent during the month
    4. **activation_date**.- when she starts being client
    5. **province**.- province (of Spain) where the client lives
    6. **phone_lines** number of phone lines the client has with the company
    7. **default_lines** number of phone lines the client failed to pay
    8. **incident** YES if the client had an incident or complaint
* spendings_month
    1. **id**.- id of the client
    2. **incoming_calls**.- number of incoming calls in all of the client's lines
    3. **outgoing_calls**.- number of outgoing calls in all of the client's lines
    4. **mb_data**.- Megabytes spent in all of the client's lines
    5. **secs_incoming_calls**.- seconds of incoming calls in all of the client's lines
    6. **secs_outgoing_calls**.- seconds of outgoing calls in all of the client's lines
* products_month
    1. **id**.- id of the client
    2. **connection**.- internet connection type hired by the client
    3. **connection_speed**.- connection speed hired by the client
    4. **TV**.- pay-television offer hired by the client
* financing_month
    1. **id**.- id of the client
    2. **financing**.- YES if the cliente has financed the purchase of any device
    3. **financ_amount** .- monthly amount the client pays for the devices she has financed
    4. **discounts**.- YES if the client benefit from any discount (marketing campaigns)
    

The target variable (¿has the client unsubscribe?) should be created comparing two consecutive months, getting the id clients of one month that are not present in the following one. We have data for the months of december and january so, we should merge the four files for each month and identify which clients has unsubscribed during december.

In [29]:
#Read december data
clients_dec = pd.read_csv('data/december/customers_december.csv', sep='|')
print('{} clients in December'.format(len(clients_dec)))
clients_dec.head()

95467 clients in December


,id,age,billing,activation_date,province,phone_lines,defaulted_lines,incident
0,1,63,216.028109,11/23/2018 08:48 AM,La Rioja,5,NaN,NaN
1,2,84,255.830842,08/22/2017 03:19 AM,Vizcaya,3,NaN,NaN
2,3,66,135.768153,12/27/2001 01:50 PM,Albacete,4,NaN,NaN
3,4,69,255.658527,08/08/2015 10:53 AM,Lugo,4,NaN,NaN
4,5,25,22.302845,08/29/1997 02:19 AM,Tarragona,2,2.0,NaN


In [10]:
spendings_dec = pd.read_csv('data/december/spending_december.csv', sep='|')
spendings_dec.head()

,id,incoming_calls,outgoing_callsl,mb_data,secs_incoming_calls,secs_outgoing_calls
0,1,110,79,10897,12806,13751
1,2,189,89,18657,6499,10862
2,3,129,30,15511,17013,16743
3,4,51,52,12670,3393,6771
4,5,183,3,23756,18436,4485


In [15]:
financing_dec = pd.read_csv('data/december/financing_december.csv', sep='|')
financing_dec.head()

,id,financing,financ_amount,discounts
0,2,NaN,NaN,YES
1,3,NaN,NaN,YES
2,7,YES,31.553269,NaN
3,11,NaN,NaN,YES
4,12,NaN,NaN,YES


In [12]:
products_dec = pd.read_csv('data/december/products_december.csv', sep='|')
products_dec.head()

,id,connection,connection_speed,TV
0,1,FIBRA,50MB,tv-futbol
1,2,FIBRA,600MB,tv-futbol
2,3,ADSL,35MB,tv-futbol
3,4,FIBRA,200MB,tv-familiar
4,5,ADSL,10MB,tv-futbol


In [19]:
#Merge the 4 december files
december = clients_dec.merge(right=spendings_dec, on='id', how='outer')
december = december.merge(right=financing_dec, on='id',how='outer')
december = december.merge(right=products_dec, on='id',how='outer')
print('The monthly file with all the information has {} columns'.format(december.shape[1]))

The monthly file with all the information has 19 columns


In [21]:
#The same read and merge process for january
clients_jan = pd.read_csv('data/january/customers_january.csv', sep='|')
spendings_jan = pd.read_csv('data/january/spending_january.csv', sep='|')
financing_jan = pd.read_csv('data/january/financing_january.csv', sep='|')
products_jan = pd.read_csv('data/january/products_january.csv', sep='|')

january = clients_jan.merge(right=spendings_jan, on='id', how='outer')
january = january.merge(right=financing_jan, on='id',how='outer')
january = january.merge(right=products_jan, on='id',how='outer')
print('The january file with all the information has {} columns'.format(january.shape[1]))

The january file with all the information has 19 columns


#### Target variable
Now we have all the information to identify the clients who leave the company in december. This will be the target variable we will try to predict.


In [22]:
december['Cancellation'] = december.id.isin(january.id)==False

In [34]:
print('{} clients leave in December. It means a churn rate of {:.2f}%'.format(december.Cancellation.sum(),december.Cancellation.sum()*100/len(december)))

7085 clients leave in December. It means a churn rate of 7.42%
